In [2]:
import geopandas as gpd
import fiona
kml_file = "../files/CTO_113-11.kml"
kml_clean_file = "../files/Legend.kml"
import pandas as pd

gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'


In [3]:
"""If you wanted to have all the folders of the kml file you can unccoment this block
and comment the one below"""

# df = pd.DataFrame()
# for layer in fiona.listlayers(kml_file):
#     s = gpd.read_file(kml_file, driver='KML', layer=layer)
#     df = pd.concat([df, s], ignore_index=True)

'If you wanted to have all the folders of the kml file you can unccoment this block\nand comment the one below'

In [4]:
# However, at this point we only want the folder with the utility poles.
df = gpd.read_file(kml_clean_file, driver='KML')

""" Since the type of the column "geometry" is Shapely.Point
    we have to change it manually."""
from shapely import wkt
df['str_geometry'] = df.geometry.apply(lambda x: wkt.dumps(x))
df['str_geometry'] = df.str_geometry.apply(lambda x: x[9:].split()[:2])
df['str_geometry'] = df.str_geometry.apply(lambda x: [x[1],x[0]])
df['latitude'] = df.str_geometry.apply(lambda x: x[0])
df['longitude'] = df.str_geometry.apply(lambda x: x[1])
df['ini'] = 2


In [14]:
""" It'd be a great idea to get the distances between every single utility pole
    before starting the actual algorithm"""
import geopandas as gpd
from geopy.distance import geodesic
from scipy.spatial.distance import pdist, squareform
coords = df[['latitude', 'longitude']].values
dist_matrix = list(squareform(pdist(coords, lambda u, v: geodesic(u, v).meters)))


In [15]:
""" Now that we have a matrix, which each element Aij, represents the distance
    from pole i to pole j, we want to associate those distances with each pole
    """
poles_names = list(df.Name)
graph = {}
for i in range(len(poles_names)):
    graph[poles_names[i]] = dist_matrix[i]


In [16]:
graph

{'POSTE_113-111': array([   0.        ,  134.93766365,  303.82107842,  722.30100232,
         793.38423912, 1135.42883848, 1280.12634803, 1415.84747675,
        1517.90847895, 1659.29504831,  944.07683777,  316.35210875,
         829.72110791,  720.49345737, 1503.65715327, 1573.87836047,
        1458.48416905, 1329.85555252, 1058.84648643,  950.73393574,
         830.3281812 , 3165.90216821, 2886.30269457, 2777.75563728,
        2670.93957278, 2843.30247012, 2707.46553061, 2580.67193842,
        2552.55820182, 3100.0713834 , 3205.29164986, 3315.76016447,
        3680.71662691, 3702.78157767, 2238.63599494, 1739.25808794,
        1468.16852908, 1242.22272742, 1110.39347578,  940.39172135,
         876.43330044,  852.66720901,  197.92738021, 4167.32326147,
        4364.94968813, 1226.40944544, 1471.18942292, 1018.83825636,
        2780.8520762 , 2790.69789808, 2793.28387831, 2751.4749939 ,
        3013.20783013, 3037.63822996, 2938.29767841, 3091.85141406,
        3202.66137752, 3215.741